<a href="https://colab.research.google.com/github/prrmzz/ElectronicComponent-SqueezeNet/blob/main/ElectronicComponentDetector_SqueezeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, roc_curve
from tensorflow.keras import layers, models
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_dir = '/content/drive/My Drive/Electronic_components'
component_folders = ['Capacitor', 'Resistor', 'IC', 'Transistor', 'Dataset_Treino']
image_size = (224, 224)  # Input size for SqueezeNet
batch_size = 32
epochs = 10

In [4]:
def is_valid_image(file_path):
    try:
        img = Image.open(file_path)
        img.verify()
        return True
    except (IOError, SyntaxError):
        return False

In [5]:
def load_image_paths_and_labels(base_dir, component_folders):
    image_paths = []
    labels = []
    label_mapping = {folder: idx for idx, folder in enumerate(component_folders)}

    for folder in component_folders:
        folder_path = os.path.join(base_dir, folder)
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if is_valid_image(file_path):
                image_paths.append(file_path)
                labels.append(label_mapping[folder])
            else:
                print(f"Invalid image file skipped: {file_path}")

    return np.array(image_paths), np.array(labels)

In [6]:
image_paths, labels = load_image_paths_and_labels(base_dir, component_folders)

In [7]:
def preprocess_image(image_path, image_size):

    image = tf.io.read_file(image_path)
    try:
        image = tf.image.decode_image(image, channels=3, expand_animations=False)
    except tf.errors.InvalidArgumentError:
        return None
    image = tf.image.resize(image, image_size)  # Resize the image to the specified size
    image = image / 255.0  # Normalize to [0, 1]
    return image

In [8]:
def create_dataset(image_paths, labels, image_size, batch_size):
    def generator_fn():
        for i in range(len(image_paths)):
            image = preprocess_image(image_paths[i], image_size)
            if image is not None:
                yield image, labels[i]

    dataset = tf.data.Dataset.from_generator(
        generator_fn,
        output_signature=(
            tf.TensorSpec(shape=image_size + (3,), dtype=tf.float32),  # Image shape
            tf.TensorSpec(shape=(), dtype=tf.int32)  # Label shape
        )
    )
    dataset = dataset.shuffle(len(image_paths)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [9]:
dataset = create_dataset(image_paths, labels, image_size, batch_size)

In [10]:
train_size = int(0.7 * len(image_paths))
val_size = int(0.15 * len(image_paths))
test_size = len(image_paths) - train_size - val_size

In [11]:
train_dataset = dataset.take(train_size)
remaining = dataset.skip(train_size)
val_dataset = remaining.take(val_size)
test_dataset = remaining.skip(val_size)

In [12]:
def build_faster_squeezenet(input_shape=(128, 128, 3), num_classes=5):
    inputs = layers.Input(shape=input_shape)

    # Initial Convolution with BatchNorm
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Fire Module with Dense and Residual Connections
    def fire_module(x, squeeze, expand):

        input_tensor = x

        # Squeeze layer
        squeeze_layer = layers.Conv2D(squeeze, (1, 1), padding='valid')(x)
        squeeze_layer = layers.BatchNormalization()(squeeze_layer)
        squeeze_layer = layers.Activation('relu')(squeeze_layer)

        # Expand layer
        expand_1x1 = layers.Conv2D(expand, (1, 1), padding='valid')(squeeze_layer)
        expand_1x1 = layers.BatchNormalization()(expand_1x1)
        expand_1x1 = layers.Activation('relu')(expand_1x1)

        expand_3x3 = layers.Conv2D(expand, (3, 3), padding='same')(squeeze_layer)
        expand_3x3 = layers.BatchNormalization()(expand_3x3)
        expand_3x3 = layers.Activation('relu')(expand_3x3)

        # Concatenate outputs
        x = layers.Concatenate()([expand_1x1, expand_3x3])

        # Skip connection (like ResNet)
        if x.shape[-1] != input_tensor.shape[-1]:
            input_tensor = layers.Conv2D(x.shape[-1], (1, 1), padding='same')(input_tensor)
            x = layers.add([x, input_tensor])

        return x

    # Block 1
    x = fire_module(x, squeeze=16, expand=32)
    x = fire_module(x, squeeze=16, expand=32)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Block 2
    x = fire_module(x, squeeze=32, expand=64)
    x = fire_module(x, squeeze=32, expand=64)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Block 3
    x = fire_module(x, squeeze=64, expand=128)
    x = layers.Dropout(0.5)(x)

    # Final Layers
    x = layers.Conv2D(num_classes, (1, 1), padding='valid')(x)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Activation('softmax')(x)

    return models.Model(inputs, outputs)

In [13]:
model = build_faster_squeezenet(input_shape=image_size + (3,), num_classes=len(component_folders))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

Epoch 1/10


In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy:.2f}")

In [ ]:
y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_pred = model.predict(test_dataset)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
for i in range(len(component_folders)):
    fpr, tpr, _ = roc_curve(y_true == i, y_pred == i)
    auc = roc_auc_score(y_true == i, y_pred == i)
    print(f"Class: {component_folders[i]}, AUC: {auc:.4f}, TPR: {tpr[1]:.4f}, FPR: {fpr[1]:.4f}")